In [3]:
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC
from datasets import load_dataset
import soundfile as sf
import torch

In [4]:
 # load model and tokenizer
 tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
 model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

In [10]:
speech, bitrate = sf.read("./sample1.flac")
print(speech.shape, bitrate)

(219040,) 16000


In [11]:
# tokenize
input_values = tokenizer(speech, return_tensors="pt", padding="longest").input_values  # Batch size 1

In [13]:
input_values.size()

torch.Size([1, 219040])

In [14]:
# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predicted_ids)

In [15]:
transcription

["GOING ALONG SLUSHY COUNTRY ROADS AND SPEAKING TO DAMP AUDIENCES IN DRAUGHTY SCHOOL ROOMS DAY AFTER DAY FOR A FORTNIGHT HE'LL HAVE TO PUT IN AN APPEARANCE AT SOME PLACE OF WORSHIP ON SUNDAY MORNING AND HE CAN COME TO US IMMEDIATELY AFTERWARDS"]

In [16]:
predicted_ids

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0, 21,  0,  0,  0,  8,  0,  0,  0, 10,
          9, 21, 21,  4,  4,  7,  7,  0, 15,  0,  0,  8,  9,  9, 21, 21,  0,  4,
          4,  0,  0,  0,  0, 12, 12,  0,  0,  0, 15, 15,  0,  0, 16,  0,  0, 12,
          0, 11,  0,  0,  0,  0, 22,  0,  4,  4,  0,  0,  0,  0, 19,  0,  0,  8,
         16, 16,  9,  9,  0,  6,  6,  0, 13,  0,  0, 22, 22,  4,  4,  0, 13, 13,
          0,  0,  0,  0,  8,  8,  7,  0,  0,  0, 14,  0,  0, 12, 12,  4,  4,  0,
          7,  9, 14, 14,  4,  4,  0, 12,  0, 23,  0,  5,  0,  7,  0, 26,  0,  0,
          0, 10,  9, 21, 21,  4,  4,  6,  6,  8,  8,  4,  4,  0, 14,  0,  0,  0,
          0,  0,  7, 17, 17,  0,  0, 23,  0,  0,  4,  0,  0,  0,  0,  0,  7, 16,
         16,  0,  0, 14,  0, 10, 10,  0,  0,  5,  9,  9,  0,  0, 19,  0,  0,  0,
          5,  5,  0,  0, 12, 12,  4,  4,  4,  0,  0, 10,  9,  0,  4,  4,  0,  0,
         14,  0, 13,  0,  0,  0,  0,  0,  7, 16, 21, 11, 11,  0,  0,  6,  0,  0,
          0, 22,  0,  4,  4,

In [18]:
# Exporting from torch to onnx produces some errors and they
# are because of how the t5-small model is coded in the transformers
# package and onnx cannot track the flow of python values and thats why
# cannot generate a proper model graph but still generates one.  
torch.onnx.export(model,
                 (input_values),
                 "model.onnx",
                 input_names=["input_ids"],
                 output_names=["logits"],
                 opset_version=12)

/usr/local/lib/python3.9/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:288: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert attn_weights.size() == (
/usr/local/lib/python3.9/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:327: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert attn_output.size() == (


In [23]:
# Converting from onnx to openvino fails because of the above mentioned reason.  
!python3 /path/to/openvino/openvino_2021.2.185/deployment_tools/model_optimizer/mo.py --input_model model.onnx

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/Users/yashbonde/Desktop/wrk/nbx/box_of_ai_tools/Natural_Language_Processing/Speech_to_Text/Wave2Vec/model.onnx
	- Path for generated IR: 	/Users/yashbonde/Desktop/wrk/nbx/box_of_ai_tools/Natural_Language_Processing/Speech_to_Text/Wave2Vec/.
	- IR output name: 	model
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	False
ONNX specific parameters:
Model Optimizer version: 	2021.2.0-1877-176bdf51370-releases/2021/2
[ ERROR ]  
Detected not satisfied dependen

The above error comes because of incorrect initialisation of variables, which can be fixed if you follow the instructions [here](https://github.com/NimbleBoxAI/hf2OpenVino). Once initialised correctly, run the instructions below.

In [ ]:
from openvino.inference_engine import IECore

In [ ]:
ie = IECore()
net = ie.read_network("./model.xml", "./model.bin")
exec_net = ie.load_network(network=net, device_name="CPU", num_requests=2)

In [ ]:
out = exec_net.infer(inputs={"input_values": input_values})["output"]

In [ ]:
transcription = [tokenizer.decode(x) for x in out]